In [ ]:
import os
# import wget
import object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-4')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    
    return detections

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
# # ### Only one Image
# IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'testing', 'download.jpg')
# img = cv2.imread(IMAGE_PATH)
# image_np = np.array(img)

# input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
# detections = detect_fn(input_tensor)


# num_detections = int(detections.pop('num_detections'))
# detections = {key: value[0, :num_detections].numpy()
#               for key, value in detections.items()}
# detections['num_detections'] = num_detections

# # detection_classes should be ints.
# detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

# label_id_offset = 1
# image_np_with_detections = image_np.copy()

# viz_utils.visualize_boxes_and_labels_on_image_array(
#             image_np_with_detections,
#             detections['detection_boxes'],
#             detections['detection_classes']+label_id_offset,
#             detections['detection_scores'],
#             category_index,
#             use_normalized_coordinates=True,
#             max_boxes_to_draw=5,
#             min_score_thresh=.8,
#             agnostic_mode=False)

# plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
# plt.show()

In [ ]:
def functionBtnDetection(): 
        aWindow.withdraw()
        ### Detection
        cap = cv2.VideoCapture(0)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        temp=''

        while cap.isOpened(): 
            ret, frame = cap.read()
            image_np = np.array(frame)

            input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
            detections = detect_fn(input_tensor)

            num_detections = int(detections.pop('num_detections'))
            detections = {key: value[0, :num_detections].numpy()
                          for key, value in detections.items()}
            detections['num_detections'] = num_detections

            # detection_classes should be ints.
            detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

            label_id_offset = 1
            image_np_with_detections = image_np.copy()

            viz_utils.visualize_boxes_and_labels_on_image_array(
                        image_np_with_detections,
                        detections['detection_boxes'],
                        detections['detection_classes']+label_id_offset,
                        detections['detection_scores'],
                        category_index,
                        use_normalized_coordinates=True,
                        max_boxes_to_draw=1,
                        min_score_thresh=.6,
                        agnostic_mode=False)
            
            cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
            
            if detections['detection_scores'][0] >= 0.90 :
                x = category_index[detections['detection_classes'][0]+1]['name']
                if x != temp :
                    pyttsx3.speak(x)
                    temp = x
                
            if cv2.waitKey(10) & 0xFF == ord('q') :
                aWindow.deiconify()
                cap.release()
                cv2.destroyAllWindows()
                break

In [ ]:
def speak():
    aWindow.withdraw()
   
    aWindow2 = Toplevel(root)
    aWindow2.title("Deep Matrix")
    # aWindow2.configure(bg="#D4E6F1")
    aWindow2.geometry('800x500')
    
    
    
    def talk():
        if my_entry.get() == "" : 
            messagebox.showinfo('Empty field' , 'Please Enter any text')
        else : 
            engine= pyttsx3.init()
            engine.say(my_entry.get())    
            engine.setProperty('rate', 100) 
            timestr = time.strftime("%Y%m%d-%H%M%S") 
            engine.save_to_file(my_entry.get(), f"{my_entry.get()[:10]}-{timestr}.mp3")
            engine.runAndWait()
            my_entry.delete(0,END)

    def backFunction():
        aWindow2.withdraw()
        aWindow.deiconify()
   
   
    image2 = ImageTk.PhotoImage(file = "dreamstime.png")
    label2 = Label(aWindow2,image=image2)
    label2.place(x=10, y=100, relwidth=1,relheight=1)
  
    style = Style()
    
    backBtn=Button(aWindow2,width=5 ,text='👈',command =backFunction)
    backBtn.pack(pady=0,padx=0 ,side=TOP)
        
    my_entry=Entry(aWindow2,width=50,  font = ("Helvetica" , 20)) 
    my_entry.pack(pady=50)
   
    my_button=Button(aWindow2 , text="talk" ,command=talk , style = 'TButton')
    my_button.pack(pady=50 ,padx=100 )
   
    style.configure('TButton', font =('calibri', 20, 'bold'),borderwidth = '4')
    style.map('TButton', foreground = [('active', '!disabled', 'green')],background = [('active', 'black')])
     


In [ ]:
# def AboutUs():
#     aWindow.withdraw()
#     aWindow2 = Toplevel(root)
#     aWindow2.title("Deep Matrix")
#     aWindow2.configure(bg="#D4E6F1")
#     aWindow2.geometry('800x500')


In [ ]:
from tkinter import *
from tkinter import messagebox

import pyttsx3
import sys
from tkinter.ttk import *
from PIL import ImageTk,Image
import uuid
import time


In [ ]:
root=Tk()
root.withdraw()

aWindow = Toplevel(root)
aWindow.title("Deep Matrix")
aWindow.geometry('945x700')


In [ ]:
image1 = PhotoImage(file = "Deaf.png")
label1 = Label(aWindow,image=image1)
label1.place(x=0, y=0, relwidth=1,relheight=1)

In [ ]:
style = Style()
style.configure('TButton', font =('Britannic Bold', 20, 'bold'),borderwidth = '9',foreground='#091885')
style.map('TButton', foreground = [('active', '!disabled', '#8311B4')],background = [('active', 'white')])

In [ ]:
btn=Button(aWindow, text="Object Detection",width=15, command=functionBtnDetection,style = 'TButton').pack(pady=100,padx=100)
btnSpeak=Button(aWindow,text="Speak", width=15,command=speak ).pack(pady=0,padx=100)
# btnAboutus=Button(aWindow,text="About Us", width=15, command=AboutUs).pack(pady=100,padx=100)

In [ ]:

# !pip install pyttsx3

In [ ]:
root.mainloop()
